In [1]:
import pandas as pd
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mrmr import mrmr_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel, chi2, mutual_info_classif
from sklearn.svm import SVC
import math
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import LeaveOneOut

In [2]:
#load the dataset from preprocessing
df = pd.read_csv('df.csv')

In [3]:
#drop the first 5 fixtures of every year
df.drop(df.index[0:50], inplace=True)

In [4]:
df = df.reset_index(drop=True)

In [5]:
df.drop(df.index[330:380], inplace=True)

In [6]:
df = df.reset_index(drop=True)

In [7]:
df.drop(df.index[660:710], inplace=True)

In [8]:
df = df.reset_index(drop=True)

In [9]:
df.drop(df.index[990:1040], inplace=True)

In [10]:
df = df.reset_index(drop=True)

In [11]:
df.drop(df.index[1320:1370], inplace=True)

In [12]:
df = df.reset_index(drop=True)

In [13]:
df.drop(df.index[1650:1700], inplace=True)

In [14]:
df = df.reset_index(drop=True)

In [15]:
df = df.drop(['home_team','away_team', 'year'], axis=1)

In [16]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,win,home_avg_goal,away_avg_goal,home_avg_pos,away_avg_pos,home_avg_pas,away_avg_pas,Home_avg_SoT,Away_avg_SoT,Home_avg_Fouls,Away_avg_Fouls,Home_avg_Cros,Away_avg_Cros,home_Value,away_Value,home_points,away_points,home_team_home_points,away_team_away_points,last_5_home_games_points,last_5_away_games_points,home_team_home_avg_goals,away_team_away_avg_goals
win,1.000000,-0.257508,0.271924,-0.233085,0.231840,-0.228265,0.244837,-0.230470,0.238862,0.105420,-0.080092,-0.072196,0.035140,-0.271868,0.287613,-0.174855,0.161271,-0.150142,0.174336,-0.206728,0.219573,-0.228035,0.246231
home_avg_goal,-0.257508,1.000000,-0.048456,0.722786,0.004304,0.718734,-0.002171,0.727733,-0.019586,-0.269887,0.034230,0.130971,-0.006862,0.742172,-0.039112,0.505420,-0.024752,0.429463,-0.019684,0.676478,-0.011466,0.870794,-0.030155
away_avg_goal,0.271924,-0.048456,1.000000,-0.020405,0.706419,-0.021788,0.701679,-0.047597,0.722124,0.046245,-0.257854,-0.006613,0.124779,-0.034467,0.736611,-0.057592,0.479145,-0.044342,0.493739,-0.024492,0.683703,-0.048038,0.834846
home_avg_pos,-0.233085,0.722786,-0.020405,1.000000,0.000670,0.941199,-0.007216,0.673849,-0.005417,-0.240137,0.010418,0.224052,0.009674,0.767642,-0.026823,0.420789,-0.006818,0.356343,0.000632,0.552321,0.001350,0.627879,-0.009548
away_avg_pos,0.231840,0.004304,0.706419,0.000670,1.000000,-0.010831,0.941358,-0.010427,0.669900,-0.004322,-0.251596,0.017004,0.221152,-0.011006,0.766815,-0.004185,0.419614,0.002452,0.437410,-0.003911,0.551507,0.000118,0.601410
home_avg_pas,-0.228265,0.718734,-0.021788,0.941199,-0.010831,1.000000,0.026631,0.614723,-0.034734,-0.197190,0.119807,0.009053,-0.171633,0.798736,-0.031438,0.411757,-0.024893,0.340561,-0.000941,0.564044,0.003534,0.624805,0.011481
away_avg_pas,0.244837,-0.002171,0.701679,-0.007216,0.941358,0.026631,1.000000,-0.038106,0.605786,0.109783,-0.202970,-0.166054,0.002443,-0.015174,0.797960,-0.014859,0.416573,-0.019913,0.441358,0.005513,0.553556,-0.020139,0.594439
Home_avg_SoT,-0.230470,0.727733,-0.047597,0.673849,-0.010427,0.614723,-0.038106,1.000000,-0.021439,-0.248858,-0.037886,0.280998,0.097362,0.647172,-0.034675,0.406199,-0.013009,0.337079,-0.009143,0.567268,-0.018779,0.627891,-0.035201
Away_avg_SoT,0.238862,-0.019586,0.722124,-0.005417,0.669900,-0.034734,0.605786,-0.021439,1.000000,-0.033329,-0.246565,0.081797,0.286187,-0.010467,0.634587,-0.032294,0.390579,-0.019923,0.413626,-0.024722,0.582716,-0.009366,0.605786
Home_avg_Fouls,0.105420,-0.269887,0.046245,-0.240137,-0.004322,-0.197190,0.109783,-0.248858,-0.033329,1.000000,0.208121,-0.380642,-0.338297,-0.208975,0.031449,-0.239753,-0.050299,-0.231593,-0.022222,-0.193743,0.027163,-0.290260,0.077818


In [17]:
df = df.drop(['home_avg_pas','away_avg_pas', 'home_team_home_points', 'away_team_away_points',
               'home_team_home_avg_goals', 'away_team_away_avg_goals'], axis=1)

In [18]:
df['win'] = df['win'].astype(int)

In [19]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(df['win'])

In [20]:
df['win'] = encoded

In [21]:
def grid(df,clf,para):
    x = df.drop(['win'], axis=1)
    
    y = df['win']
    
    selected_features = mrmr_classif(x, y, K = 35)
    
    x_mrmr = df[selected_features]
    
    cv = LeaveOneOut()
    sm = SVMSMOTE()
    kbest = SelectKBest()
    
    classifier = clf 
    parameters = para
    
    
    x_train, x_test, y_train, y_test = train_test_split(x_mrmr, y, train_size=0.8, random_state=88)
    
    pipe= Pipeline(steps=[('sm',sm), ('kbest',kbest), ('classifier', classifier)])
    
    pipe.fit(x_train, y_train)
    
    cv= GridSearchCV(pipe, parameters, scoring='balanced_accuracy', verbose=1, n_jobs=-1, cv=cv)
    
    grid_result=cv.fit(x_train,y_train)
    
    
    return grid_result.best_score_, grid_result.best_params_

In [42]:
clf = [(RandomForestClassifier(), {'classifier__class_weight':['balanced'],
                                                 'classifier__bootstrap': [True],
                                                 'classifier__max_depth': [5, 10, 20],
                                                 'classifier__max_features': ['auto'],
                                                 'classifier__min_samples_leaf': [1, 2, 3],
                                                 'classifier__min_samples_split': [2, 5, 10],
                                                 'classifier__n_estimators': [100, 200, 300]}),
       (KNeighborsClassifier(), {'classifier__n_neighbors':[1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]
                                }),
        (GradientBoostingClassifier(), {'classifier__loss':['deviance', 'exponential'],
                                                  'classifier__n_estimators' : [30, 60, 100, 150, 200],
                                                  'classifier__learning_rate': [0.01, 0.05, 0.1, 0.15],
                                                  'classifier__max_depth': [1, 6, 10, 15, 20],
                                                  }),
        (SVC(), {'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
                          'classifier__C': [1, 10, 100, 1000],
                          'classifier__class_weight':['balanced'],
                          'classifier__gamma' : [0.001, 0.01, 0.1, 1]})]

In [2]:
output={}
for i in clf:
    print(i[0])
    bestscore,bestparams = grid(df,i[0],i[1])
    output.update({i[0]:[bestscore,bestparams]})    